## Problem Set 3: Machine Combinations

_csc427, semester 222
<br>
university of miami
<br>
date: 9 feb 2022_


#  Student name: Diep Vu

---

### Preliminary Code

This is the code implementing the computation of a Deterministic Finite State Automata, DFA.

Also a class to place the DFA in a test jig and test it.


In [119]:
class SimpleFiniteAutomata:
    
    def __init__(self,fa_description):
        self.fa = fa_description
        self.state = self.fa['start']
        
    def one_step(self,symbol):
        assert symbol in self.fa['alphabet']
        assert (self.state,symbol) in self.fa['transitions']
        return self.fa['transitions'][(self.state,symbol)]
        
    def compute(self,string,verbose=False):
        self.state = self.fa['start']
        if verbose:
            print(f'input: |{string}|')
        for symbol in string:
            s = self.one_step(symbol)
            if verbose:
                print(f'({self.state},{symbol}) -> {s}')
            self.state = s
        if verbose:
            s = ('reject','accept')[self.state in self.fa['accept']]
            print(s)
        return self.state in self.fa['accept']

    
#end class SimpleFiniteAutomata

In [120]:
class SFA_Test:
    
    def __init__(self, fa_description):
        self.sfa = None
        self.test_vector = None
        if fa_description:
            self.sfa = SimpleFiniteAutomata(fa_description)
  
    def test_v(self, test_vector, label, verbose=False):
        if not self.sfa: return
    
        tv_true, tv_false = test_vector
        correct = 0 

        print(f'*** testing {label}')
        
        for string in tv_true:
            if verbose: print()
            if self.sfa.compute(string,verbose):
                correct += 1
            else:
                print(f'should accept but does not: |{string}| ')
        print(f'\t{correct} correctly accepted out of {len(tv_true)} strings')
        passed = correct == len(tv_true)

        correct = 0
        for string in tv_false:
            if verbose: print()
            if not self.sfa.compute(string,verbose):
                correct += 1
            else:
                print(f'should reject but does not: |{string}| ')
 
        print(f'\t{correct} correctly rejected out of {len(tv_false)} strings')
 
        passed = passed and (correct == len(tv_false))
        if passed:
            print(f'*** PASSES\n')
        else:
            print(f'*** FAILS\n')
        return passed

# end class SFA_Tests 

### Exercise A

Write the code that creates new machines from old machines.

- union: takes two machines and returns a machine accepting the union of the languages.
- intersect: takes two machines and returns a machine accepting the intersection of the languages.
- complement: takes a machine and returns a machine accepting the complement of the language.

To implement, you implement the statick methods, 

- cross_states: computes the cartesian product of two sets
- cross_deltas: the heart of the code. to create the transition map that simultaneously follows
  the transtions of the two parallel machines
 
then write the code that calculates the set of accept states.

Note that this class contains all static methods. A static method is just a simple define. However 
the name of the define is not placed in the global namespace, but in the namespace that is created
by the class construct, with name that of the class. To invoke the method, use the namespace name
before the dot, e.g. SFA_Ops.union().


In [121]:

class SFA_Ops:
    
    @staticmethod
    def cross_states(set1, set2):
        cp = set()
        
        # code to make the cartesian product of set1 by set2
        for a in set1:
            for b in set2:
                cp.add((a, b))
        
        return cp
    
    @staticmethod
    def cross_deltas(state, alphabet, d1, d2):
        d = {} # an empty dictionary
        
        # code to make the cartesian product transition function from
        # the transition functions d1 and d2
        
        for states in state:
            for symbol in alphabet:
                d[(states[0], states[1]), symbol] = (d1[(states[0], symbol)],  d2[(states[1], symbol)])

        return d      
       
    @staticmethod
    def cross_machine(fa1, fa2):
        assert fa1['alphabet'] == fa2['alphabet']
        """
        returns a new dictionary; however the elements are not copied
        """
        states = SFA_Ops.cross_states(
                    fa1['states'],
                    fa2['states'])
        d = SFA_Ops.cross_deltas(states,
                    fa1['alphabet'],
                    fa1['transitions'],
                    fa2['transitions'])
                
        fa = {
            'states': states,
            'alphabet': fa1['alphabet'],
            'transitions': d,
            'start': (fa1['start'],fa2['start']),
            'accept': set()
        }
        return fa
    
    @staticmethod
    def intersect(fa_o,fa_i):
        desc = SFA_Ops.cross_machine(fa_o,fa_i)
        accept = set()
        
        # create the set of accept states for the intersection of 
        # languages accepted by fa_o and fa_i
        
        accept = SFA_Ops.cross_states(fa_o['accept'], fa_i['accept']) 
        
        desc['accept'] = accept
        return desc
  
    @staticmethod
    def union(fa_o,fa_u):
        desc = SFA_Ops.cross_machine(fa_o,fa_u)
        accept = set()
        
        # create the set of accept states for the union of 
        # languages accepted by fa_o and fa_i
        
        accept = SFA_Ops.cross_states(fa_o['accept'], fa_u['states'])
        
        temp = SFA_Ops.cross_states(fa_o['states'], fa_u['accept'])
        
        for i in temp:
            accept.add(i)
        
        desc['accept'] = accept
        return desc
    
    @staticmethod
    def complement(fa_c):
        desc = fa_c.copy() # a shallow copy, to avoid aliasing two machines
        accept = set()
        
        # create the set of accept states for the complement of 
        # language accepted by fa_c
        
        accept = desc['states'].copy()
        for i in desc['accept']:
            accept.remove(i)
        
        desc['accept'] = accept
        return desc

# end class SFA_Ops

### Exercise B

Test your code. This gives an example of how the SFA_Ops class and SFA_Test class are used. 

Because the methonds in SFA_Ops are marked @staticmethod, there is no need to instatiate an 
object of that type to use the methods of the class. It suffices to give the name of the class
before the dot.

You might also wish to challenge your code by trying additional examples; for instance, a
language intersected with itself should be the "accept none" language.


In [122]:
m2 = {
    'states':{'q1','q2'},
    'alphabet':{'0','1'},
    'transitions':{
        ('q1','0'):'q1',('q1','1'):'q2',
        ('q2','0'):'q1',('q2','1'):'q2',
    },
    'start':'q1',
    'accept':{'q2'}
}

m2_complement = SFA_Ops.complement(m2)
print (m2_complement)
accept_all = SFA_Ops.union(m2,m2_complement)
print(accept_all)

m = SFA_Test(accept_all)
m_test = (['','0','1','00','10','01','11','101','0011'],[])
m.test_v(m_test,'accept all strings')


{'states': {'q1', 'q2'}, 'alphabet': {'0', '1'}, 'transitions': {('q1', '0'): 'q1', ('q1', '1'): 'q2', ('q2', '0'): 'q1', ('q2', '1'): 'q2'}, 'start': 'q1', 'accept': {'q1'}}
{'states': {('q2', 'q1'), ('q1', 'q1'), ('q2', 'q2'), ('q1', 'q2')}, 'alphabet': {'0', '1'}, 'transitions': {(('q2', 'q1'), '0'): ('q1', 'q1'), (('q2', 'q1'), '1'): ('q2', 'q2'), (('q1', 'q1'), '0'): ('q1', 'q1'), (('q1', 'q1'), '1'): ('q2', 'q2'), (('q2', 'q2'), '0'): ('q1', 'q1'), (('q2', 'q2'), '1'): ('q2', 'q2'), (('q1', 'q2'), '0'): ('q1', 'q1'), (('q1', 'q2'), '1'): ('q2', 'q2')}, 'start': ('q1', 'q1'), 'accept': {('q2', 'q1'), ('q2', 'q2'), ('q1', 'q1')}}
*** testing accept all strings
	9 correctly accepted out of 9 strings
	0 correctly rejected out of 0 strings
*** PASSES



True

### Exercise C

From the class text, do problems 1.4, a, c, e and f. Please do these by creating simpler FA's and using
SFA_Ops.union and SFA_Ops.complement to complete the construction. 

- Language 1.4a must use intersect
- Langauge 1.4c must use intersect
- Language 1.4e must use intersect
- Language 1.4f must use intersect and complement.



In [123]:
# {w| w has at least three a’s and at least two b’s}

at_least_three_a = {
    
    # write code here
    
    'states':{'q0','q1','q2','q3'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','b'):'q0', ('q0','a'):'q1',
        ('q1','b'):'q1', ('q1','a'):'q2',
        ('q2','b'):'q2', ('q2','a'):'q3',
        ('q3','b'):'q3', ('q3','a'):'q3'
    },
    'start':'q0',
    'accept': {'q3'}
    
}

at_least_two_b = {
 
    # write code here
    
    'states':{'q0','q1','q2'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q0', ('q0','b'):'q1',
        ('q1','a'):'q1', ('q1','b'):'q2',
        ('q2','a'):'q2', ('q2','b'):'q2'
    },
    'start':'q0',
    'accept': {'q2'}
}

ex_1_4a = SFA_Ops.intersect(at_least_three_a, at_least_two_b)
ex_1_4a_test = (['aaabb','ababa'],['','aaa','bb','aabb','aaab'])




# {w| w has an even number of a’s and one or two b’s}

even_number_of_a = {
 
    # write code here
    
    'states':{'q0','q1'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q1', ('q0','b'):'q0',
        ('q1','a'):'q0', ('q1','b'):'q1',
    },
    'start':'q0',
    'accept': {'q0'}
}

one_or_two_b = {
 
    # write code here
    
    'states':{'q0','q1','q2','q3'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q0', ('q0','b'):'q1',
        ('q1','a'):'q1', ('q1','b'):'q2',
        ('q2','a'):'q2', ('q2','b'):'q3',
        ('q3','a'):'q3', ('q3','b'):'q3'
    },
    'start':'q0',
    'accept': {'q1','q2'}
    
}

ex_1_4c = SFA_Ops.intersect(even_number_of_a, one_or_two_b)
ex_1_4c_test = (['b','bb','aba','abaaba'],['','aa','bbb','aaabb'])



# {w| w starts with an a and has at most one b}

starts_with_an_a = {
  
    # write code here
    
    'states':{'q0','q1','q2'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q1', ('q0','b'):'q2',
        ('q1','a'):'q1', ('q1','b'):'q1',
        ('q2','a'):'q2', ('q2','b'):'q2'
    },
    'start':'q0',
    'accept': {'q1'}
    
}

has_at_most_one_b = {
 
    # write code here
    
    'states':{'q0','q1','q2'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q0', ('q0','b'):'q1',
        ('q1','a'):'q1', ('q1','b'):'q2',
        ('q2','a'):'q2', ('q2','b'):'q2'
    },
    'start':'q0',
    'accept': {'q0','q1'}
    
}

ex_1_4e = SFA_Ops.intersect(starts_with_an_a, has_at_most_one_b)
ex_1_4e_test = (['a','ab','aab','aba'],['','b','abb'])


#  {w| w has an odd number of a’s and ends with a b}

ends_with_a_b = {
 
    # write code here
     
    'states':{'q0','q1','q2'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q1', ('q0','b'):'q0',
        ('q1','a'):'q1', ('q1','b'):'q2',
        ('q2','a'):'q1', ('q2','b'):'q0'
    },
    'start':'q0',
    'accept': {'q2'}
}

odd_number_of_a = SFA_Ops.complement(even_number_of_a)

ex_1_4f = SFA_Ops.intersect(odd_number_of_a, ends_with_a_b)
ex_1_4f_test = (['ab','aaab','ababab'],['','a','aab','aaba'])



In [124]:

fas = [
    (ex_1_4a,ex_1_4a_test,'1.4a'),
    (ex_1_4c,ex_1_4c_test,'1.4c'),
    (ex_1_4e,ex_1_4e_test,'1.4e'),
    (ex_1_4f,ex_1_4f_test,'1.4f')
]

passed = True
for fa in fas:
    m = SFA_Test(fa[0])
    passed = passed and m.test_v(fa[1],fa[2])
if passed:
    print(f'\n*** all {len(fas)} machines are correct')
else:
    print(f'\n*** not all machines are correct')

*** testing 1.4a
	2 correctly accepted out of 2 strings
	5 correctly rejected out of 5 strings
*** PASSES

*** testing 1.4c
	4 correctly accepted out of 4 strings
	4 correctly rejected out of 4 strings
*** PASSES

*** testing 1.4e
	4 correctly accepted out of 4 strings
	3 correctly rejected out of 3 strings
*** PASSES

*** testing 1.4f
	3 correctly accepted out of 3 strings
	4 correctly rejected out of 4 strings
*** PASSES


*** all 4 machines are correct


### Exercise D

From the class text, do problems 1.5, c, d, e and f. Please do these by simpler FA's and using
SFA_Ops.union, SFA_Ops.intersect and SFA_Ops.complement to complete the construction. 

- Language 1.5c must use intersect and may use complement
- Langauge 1.5d must use complement
- Language 1.5e must use complement
- Language 1.5f must use union and complement.


In [125]:
# {w| w contains neither the substrings ab nor ba}

contains_ab = {
  
    # write code here
      
    'states':{'q0','q1','q2'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q1', ('q0','b'):'q0',
        ('q1','a'):'q1', ('q1','b'):'q2',
        ('q2','a'):'q2', ('q2','b'):'q2'
    },
    'start':'q0',
    'accept': {'q2'}
    
}
contains_ba = {
 
    # write code here
          
    'states':{'q0','q1','q2'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q0', ('q0','b'):'q1',
        ('q1','a'):'q2', ('q1','b'):'q1',
        ('q2','a'):'q2', ('q2','b'):'q2'
    },
    'start':'q0',
    'accept': {'q2'}
    
}
ex_1_5c = SFA_Ops.intersect(SFA_Ops.complement(contains_ab), SFA_Ops.complement(contains_ba))
ex_1_5c_test = (['','a','b','aaa','bbb'],['ab','ba','aaab','bbba'])



# {w| w is any string not in a*b*}

a_star_b_star = {
 
    # write code here
    
    'states':{'q0','q1','q2'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q0', ('q0','b'):'q1',
        ('q1','a'):'q2', ('q1','b'):'q1',
        ('q2','a'):'q2', ('q2','b'):'q2'
    },
    'start':'q0',
    'accept': {'q0','q1'}
    
}
ex_1_5d = SFA_Ops.complement(a_star_b_star)
ex_1_5d_test = (['ba','aba'],['','a','b','bb','aabbb'])



# {w| w is any string not in (ab+)*}

ab_plus_star = {
 
    # write code here
    
    'states':{'q0','q1','q2','q3'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q1', ('q0','b'):'q3',
        ('q1','a'):'q3', ('q1','b'):'q2',
        ('q2','a'):'q1', ('q2','b'):'q2',
        ('q3','a'):'q3', ('q3','b'):'q3'
    },
    'start':'q0',
    'accept': {'q0','q2'}
    
}
ex_1_5e = SFA_Ops.complement(ab_plus_star)
ex_1_5e_test = (['a','b','aa','aaba'],['','ab','abbb','abbbab'])



# {w|w is any string not in a* U b*}

a_star = {
 
    # write code here
    
    'states':{'q0','q1'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','a'):'q0', ('q0','b'):'q1',
        ('q1','a'):'q1', ('q1','b'):'q1'
    },
    'start':'q0',
    'accept': {'q0'}
    
}
b_star = {
 
    # write code here
    
    'states':{'q0','q1'},
    'alphabet':{'a','b'},
    'transitions':{
        ('q0','b'):'q0', ('q0','a'):'q1',
        ('q1','a'):'q1', ('q1','b'):'q1'
    },
    'start':'q0',
    'accept': {'q0'}
    
}
ex_1_5f = SFA_Ops.complement(SFA_Ops.union(a_star, b_star))
ex_1_5f_test = (['ab','ba','baaaa','aaaaaab'],['','a','b','aaa','bbbb'])




In [126]:

fas = [
    (ex_1_5c,ex_1_5c_test,'1.5c'),
    (ex_1_5d,ex_1_5d_test,'1.5d'),
    (ex_1_5e,ex_1_5e_test,'1.5e'),
    (ex_1_5f,ex_1_5f_test,'1.5f')
]

passed = True
for fa in fas:
    m = SFA_Test(fa[0])
    passed = passed and m.test_v(fa[1],fa[2])
if passed:
    print(f'\n*** all {len(fas)} machines are correct')
else:
    print(f'\n*** not all machines are correct')


*** testing 1.5c
	5 correctly accepted out of 5 strings
	4 correctly rejected out of 4 strings
*** PASSES

*** testing 1.5d
	2 correctly accepted out of 2 strings
	5 correctly rejected out of 5 strings
*** PASSES

*** testing 1.5e
	4 correctly accepted out of 4 strings
	4 correctly rejected out of 4 strings
*** PASSES

*** testing 1.5f
	4 correctly accepted out of 4 strings
	5 correctly rejected out of 5 strings
*** PASSES


*** all 4 machines are correct
